<a href="https://colab.research.google.com/github/vedanthshenoy/Sales-Analysis-Prediction-and-Recommendation/blob/main/shop_user_correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/shopdata.csv',encoding= 'unicode_escape')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [ ]:
data.drop(columns=['InvoiceNo', 'StockCode', 'UnitPrice', 'Country','InvoiceDate'],inplace=True)
data=data.dropna(how='any')
data.head()

,Description,Quantity,CustomerID
0,WHITE HANGING HEART T-LIGHT HOLDER,6,17850.0
1,WHITE METAL LANTERN,6,17850.0
2,CREAM CUPID HEARTS COAT HANGER,8,17850.0
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,17850.0
4,RED WOOLLY HOTTIE WHITE HEART.,6,17850.0


In [ ]:
buys = pd.DataFrame(data.groupby('CustomerID')['Quantity'].mean())  
  
buys['Total Quantity'] = pd.DataFrame(data.groupby('CustomerID')['Quantity'].count()) 
  
buys.head()

,Quantity,Total Quantity
CustomerID,,
12346.0,0.000000,2
12347.0,13.505495,182
12348.0,75.516129,31
12349.0,8.643836,73
12350.0,11.588235,17


In [ ]:
usermat = data.pivot_table(index ='Description', 
              columns ='CustomerID', values ='Quantity') 
  
usermat.head(5)

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,12357.0,12358.0,12359.0,12360.0,12361.0,12362.0,12363.0,12364.0,12365.0,12367.0,12370.0,12371.0,12372.0,12373.0,12374.0,12375.0,12377.0,12378.0,12379.0,12380.0,12381.0,12383.0,12384.0,12386.0,12388.0,12390.0,12391.0,12393.0,12394.0,12395.0,...,18231.0,18232.0,18233.0,18235.0,18236.0,18237.0,18239.0,18240.0,18241.0,18242.0,18245.0,18246.0,18248.0,18249.0,18250.0,18251.0,18252.0,18255.0,18256.0,18257.0,18259.0,18260.0,18261.0,18262.0,18263.0,18265.0,18268.0,18269.0,18270.0,18272.0,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
Description,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4 PURPLE FLOCK DINNER CANDLES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50'S CHRISTMAS GIFT BAG LARGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DOLLY GIRL BEAKER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
I LOVE LONDON MINI BACKPACK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
I LOVE LONDON MINI RUCKSACK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We take a particular user with user ID 12359.0

In [ ]:
user = usermat[ 12359.0]
user

Description
 4 PURPLE FLOCK DINNER CANDLES       NaN
 50'S CHRISTMAS GIFT BAG LARGE       NaN
 DOLLY GIRL BEAKER                  12.0
 I LOVE LONDON MINI BACKPACK         4.0
 I LOVE LONDON MINI RUCKSACK         NaN
                                    ... 
ZINC T-LIGHT HOLDER STARS SMALL      NaN
ZINC TOP  2 DOOR WOODEN SHELF        NaN
ZINC WILLIE WINKIE  CANDLE STICK     NaN
ZINC WIRE KITCHEN ORGANISER          NaN
ZINC WIRE SWEETHEART LETTER TRAY     NaN
Name: 12359.0, Length: 3896, dtype: float64

In [ ]:
similar_to_user = usermat.corrwith(user)
corr_user = pd.DataFrame(similar_to_user, columns =['Correlation'])
corr_user = corr_user.join(buys['Total Quantity'])  
corr_user.dropna(inplace = True)
corr_user.head()

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,Correlation,Total Quantity
CustomerID,,
12347.0,0.784048,182
12349.0,0.928071,73
12352.0,0.906052,95
12354.0,0.983424,58
12355.0,1.000000,13


All these are similar users to userID 12359.0

In [ ]:
corr_user[corr_user['Total Quantity']>200].sort_values('Correlation', ascending = False).head(10) 

,Correlation,Total Quantity
CustomerID,,
12359.0,1.000000,254
12839.0,1.000000,316
12662.0,0.996657,232
15544.0,0.987969,261
14291.0,0.980795,238
14132.0,0.974365,201
13018.0,0.970852,459
13777.0,0.970725,219
12720.0,0.968597,359
